In [4]:
import pandas as pd
import numpy as np
import model
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import classification_report
from collections import defaultdict
from tensorflow import keras
import tensorflow as tf
from model import mobileHART_XS

In [2]:
! pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", usecols=columns)

print(df.head())

   Accel X (g)  Accel Y (g)  Accel Z (g)  Gyro X (°/s)  Gyro Y (°/s)  \
0    -0.173760    -0.137676     2.107255     -0.764989     -0.845309   
1    -0.160640     0.305352     2.080949      0.173658     -0.265426   
2    -1.884325    -2.219527     3.538072     -0.707602     -1.429224   
3    -0.810081    -1.515637    -1.006428      0.030364     -0.102753   
4     0.918867    -1.418559    -1.139802     -0.310511      0.014580   

   Gyro Z (°/s) Activity_Label  
0      0.134371        Cycling  
1      1.160738        Cycling  
2     -4.425213        Cycling  
3     -0.245121        Cycling  
4      0.228318        Cycling  


In [4]:
labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]
data = df[feature_columns].values

In [5]:
input_shape = (128, 6)
activityCount = 10
MobileHART = mobileHART_XS(input_shape, activityCount)

2024-07-18 21:10:02.573816: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-18 21:10:02.574004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-18 21:10:02.589937: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
def create_overlapping_windows(data, window_size, step_size, features):
    segments = []
    for start in range(0, len(data) - window_size + 1, step_size):
        segment = data[start:start + window_size, :features]
        segments.append(segment)
    return np.array(segments)

window_size = 128
step_size = 64
num_features = 6

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

def assign_labels_to_windows(labels, window_size, step_size):
    window_labels = []
    for start in range(0, len(labels) - window_size + 1, step_size):
        window_label = np.bincount(labels[start:start + window_size]).argmax()
        window_labels.append(window_label)
    return np.array(window_labels)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=21)

In [8]:
MobileHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
import tensorflow.keras as keras

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,        
    restore_best_weights=True  
)

In [10]:
history = MobileHART.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/100


I0000 00:00:1721351431.681234  198587 service.cc:145] XLA service 0x7fbc4402b4e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721351431.681269  198587 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
I0000 00:00:1721351431.681274  198587 service.cc:153]   StreamExecutor device (1): Quadro P6000, Compute Capability 6.1
2024-07-18 21:10:32.438938: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1721351433.363244  198587 random_ops.cc:59] Warning: Using tf.random.uniform with XLA compilation will ignore seeds; consider using tf.random.stateless_uniform instead if reproducible behavior is desired. functional_1_1/AccMHA_0_1/drop_path_1_1/random_uniform/RandomUniform
2024-07-18 21:10:35.020489: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN versi

   8/3972 ━━━━━━━━━━━━━━━━━━━━ 1:00 15ms/step - accuracy: 0.0667 - loss: 2.3279      

I0000 00:00:1721351495.117388  198587 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3970/3972 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5241 - loss: 1.1313

I0000 00:00:1721351563.345047  199162 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_63', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1721351565.199343  199165 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_63', 504 bytes spill stores, 500 bytes spill loads

I0000 00:00:1721351566.176198  199166 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721351566.628372  199160 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_65', 24 bytes spill stores, 24 bytes spill loads

I0000 00:00:1721351567.213165  199165 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_65', 504 bytes spill stores, 500 bytes spill loads

I0000 00:00:1721351567.224456  199157

3972/3972 ━━━━━━━━━━━━━━━━━━━━ 199s 29ms/step - accuracy: 0.5242 - loss: 1.1310 - val_accuracy: 0.8895 - val_loss: 0.3494
Epoch 2/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.8976 - loss: 0.3110 - val_accuracy: 0.9335 - val_loss: 0.2171
Epoch 3/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.9239 - loss: 0.2357 - val_accuracy: 0.9621 - val_loss: 0.1358
Epoch 4/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.9403 - loss: 0.1863 - val_accuracy: 0.9523 - val_loss: 0.1733
Epoch 5/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.9468 - loss: 0.1709 - val_accuracy: 0.9549 - val_loss: 0.1489
Epoch 6/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.9539 - loss: 0.1485 - val_accuracy: 0.9746 - val_loss: 0.0817
Epoch 7/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 54s 14ms/step - accuracy: 0.9584 - loss: 0.1349 - val_accuracy: 0.9736 - val_loss: 0.0816
Epoch 8/100
3972/3972 ━━━━━━━━━━━━━━━━━━━━ 55s 14ms/step - accuracy: 0.9604 - lo

In [11]:
performance = MobileHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

I0000 00:00:1721352381.094130  199993 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721352381.977934  199997 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_49', 120 bytes spill stores, 128 bytes spill loads

I0000 00:00:1721352385.212042  199991 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721352385.469546  199988 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_91', 120 bytes spill stores, 128 bytes spill loads



311/311 ━━━━━━━━━━━━━━━━━━━━ 14s 12ms/step - accuracy: 0.9739 - loss: 0.0833
Test Loss: 0.08163713663816452, Test Accuracy: 0.973615288734436


In [12]:
y_pred_prob = MobileHART.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

report = classification_report(y_test, y_pred, target_names=[f"Class {i}" for i in range(activityCount)])

print(report)

311/311 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step
              precision    recall  f1-score   support

     Class 0       1.00      1.00      1.00       980
     Class 1       1.00      1.00      1.00       945
     Class 2       1.00      1.00      1.00      1012
     Class 3       0.99      1.00      1.00       985
     Class 4       1.00      0.99      0.99       989
     Class 5       0.96      0.97      0.96       987
     Class 6       0.89      0.94      0.92      1022
     Class 7       0.98      0.90      0.94       990
     Class 8       0.99      0.95      0.97      1002
     Class 9       0.94      0.99      0.96      1018

    accuracy                           0.97      9930
   macro avg       0.97      0.97      0.97      9930
weighted avg       0.97      0.97      0.97      9930



  # Testing the trained model

In [13]:
import pandas as pd
import numpy as np

test_columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']

test_df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=test_columns)

test_labels = pd.factorize(test_df['Activity_Label'])[0]

test_feature_columns = test_df.columns[:-1]
test_data = test_df[test_feature_columns].values

test_reshaped_data = create_overlapping_windows(test_data, window_size, step_size, num_features)

test_window_labels = assign_labels_to_windows(test_labels, window_size, step_size)

test_performance = MobileHART.evaluate(test_reshaped_data, test_window_labels)
print(f"Test Loss: {test_performance[0]}, Test Accuracy: {test_performance[1]}")

1543/1552 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1186 - loss: 164310.1406

I0000 00:00:1721352548.815953  200226 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721352549.746662  200232 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721352551.118471  200234 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_91', 300 bytes spill stores, 184 bytes spill loads

I0000 00:00:1721352552.014214  200224 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_91', 272 bytes spill stores, 280 bytes spill loads



1552/1552 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - accuracy: 0.1185 - loss: 164711.5312
Test Loss: 226776.5625, Test Accuracy: 0.10866937786340714


In [14]:
test_predictions = MobileHART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))

1552/1552 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4964
           1       0.00      0.00      0.00      4965
           2       0.00      0.00      0.00      4965
           3       0.00      0.00      0.00      4964
           4       0.00      0.00      0.00      4965
           5       0.00      0.00      0.00      4965
           6       0.00      0.00      0.00      4964
           7       0.00      0.00      0.00      4965
           8       0.10      0.98      0.19      4964
           9       0.36      0.11      0.17      4965

    accuracy                           0.11     49646
   macro avg       0.05      0.11      0.04     49646
weighted avg       0.05      0.11      0.04     49646



/home/jupyter-dpemmasa/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-dpemmasa/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/jupyter-dpemmasa/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [15]:
label_counts = defaultdict(int)
total_counts = defaultdict(int)

for i in range(len(test_window_labels)):
    actual_label = test_window_labels[i]
    predicted_label = test_predictions[i]
    label_counts[f"actual_{actual_label}_predicted_{predicted_label}"] += 1
    total_counts[f"actual{actual_label}"] += 1

data = []
for label, count in label_counts.items():
    actual_label = label.split("_")[1]
    total = total_counts[f"actual{actual_label}"]
    ratio = count / total if total > 0 else 0
    data.append([label, count, total, ratio])

results_df = pd.DataFrame(data, columns=["Label", "Count", "Total", "Ratio"])
results_df.sort_values(by="Ratio", ascending=False, inplace=True)
print(results_df)

                   Label  Count  Total     Ratio
1   actual_1_predicted_8   4964   4965  0.999799
2   actual_2_predicted_8   4964   4965  0.999799
0   actual_0_predicted_8   4963   4964  0.999799
4   actual_4_predicted_8   4963   4965  0.999597
3   actual_3_predicted_8   4962   4964  0.999597
15  actual_8_predicted_8   4844   4964  0.975826
5   actual_5_predicted_8   4644   4965  0.935347
11  actual_7_predicted_8   4471   4965  0.900504
8   actual_6_predicted_8   4402   4964  0.886785
17  actual_9_predicted_8   4195   4965  0.844914
18  actual_9_predicted_9    551   4965  0.110977
9   actual_6_predicted_9    420   4964  0.084609
12  actual_7_predicted_9    336   4965  0.067674
19  actual_9_predicted_3    219   4965  0.044109
7   actual_5_predicted_3    194   4965  0.039074
13  actual_7_predicted_3    158   4965  0.031823
10  actual_6_predicted_3    142   4964  0.028606
6   actual_5_predicted_9    127   4965  0.025579
14  actual_8_predicted_9    110   4964  0.022160
16  actual_8_predict

# Mobile HART Trained on HAR and Inertia dataset finetuning on 30% of the Data

In [16]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']

df1 = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)

labels = pd.factorize(df1['Activity_Label'])[0]

feature_columns = df1.columns[:-1] 
data = df1[feature_columns].values

In [17]:
window_size = 128 
step_size = 64    
num_features = 6 

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 6)
activityCount = len(np.unique(labels))

In [18]:
MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [19]:
history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200


I0000 00:00:1721353057.954028  200435 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721353058.076380  200433 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 436 bytes spill stores, 436 bytes spill loads

I0000 00:00:1721353059.103706  200434 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 440 bytes spill stores, 440 bytes spill loads

I0000 00:00:1721353064.584312  200425 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32181', 84 bytes spill stores, 84 bytes spill loads

I0000 00:00:1721353064.939872  200433 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32181', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1721353064.9677

  1/621 ━━━━━━━━━━━━━━━━━━━━ 13:50:51 80s/step - accuracy: 0.0781 - loss: 2.3020

I0000 00:00:1721353097.117896  198587 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 4 bytes spill stores, 4 bytes spill loads



619/621 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6722 - loss: 0.7622

I0000 00:00:1721353124.187476  200631 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 432 bytes spill stores, 432 bytes spill loads

I0000 00:00:1721353124.256066  200634 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 396 bytes spill stores, 396 bytes spill loads

I0000 00:00:1721353124.415139  200640 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 36 bytes spill stores, 24 bytes spill loads

I0000 00:00:1721353126.905367  200637 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721353127.711774  200638 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721353129.48372

621/621 ━━━━━━━━━━━━━━━━━━━━ 156s 121ms/step - accuracy: 0.6729 - loss: 0.7607 - val_accuracy: 0.6823 - val_loss: 1.0688
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9469 - loss: 0.1628 - val_accuracy: 0.9523 - val_loss: 0.1567
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9657 - loss: 0.1147 - val_accuracy: 0.8990 - val_loss: 0.2901
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9660 - loss: 0.1147 - val_accuracy: 0.9769 - val_loss: 0.0775
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9758 - loss: 0.0817 - val_accuracy: 0.9736 - val_loss: 0.0826
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9764 - loss: 0.0854 - val_accuracy: 0.9756 - val_loss: 0.0802
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9739 - loss: 0.0950 - val_accuracy: 0.9770 - val_loss: 0.0691
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9785 - loss: 0.0728 - va

In [20]:
performance = MHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9815 - loss: 0.0632
Test Loss: 0.06462661921977997, Test Accuracy: 0.9815710186958313


In [21]:
test_predictions = MHART.predict(test_reshaped_data)
test_predictions = np.argmax(test_predictions, axis=1)

print(classification_report(test_window_labels, test_predictions))

1552/1552 ━━━━━━━━━━━━━━━━━━━━ 12s 6ms/step
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4964
           1       1.00      1.00      1.00      4965
           2       1.00      1.00      1.00      4965
           3       1.00      1.00      1.00      4964
           4       1.00      0.99      1.00      4965
           5       0.92      0.99      0.95      4965
           6       0.99      0.94      0.96      4964
           7       0.99      0.98      0.98      4965
           8       1.00      0.93      0.96      4964
           9       0.94      1.00      0.97      4965

    accuracy                           0.98     49646
   macro avg       0.98      0.98      0.98     49646
weighted avg       0.98      0.98      0.98     49646



In [22]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)

labels = pd.factorize(df['Activity_Label'])[0]

feature_columns = df.columns[:-1]  
data = df[feature_columns].values

In [23]:
window_size = 128 
step_size = 64    
num_features = 6

reshaped_data = create_overlapping_windows(data, window_size, step_size, num_features)

window_labels = assign_labels_to_windows(labels, window_size, step_size)

input_shape = (128, 6)
activityCount = len(np.unique(labels))  

MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200
  4/621 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - accuracy: 0.1514 - loss: 2.1984    

I0000 00:00:1721353550.184777  198588 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 4 bytes spill stores, 4 bytes spill loads



621/621 ━━━━━━━━━━━━━━━━━━━━ 130s 99ms/step - accuracy: 0.7086 - loss: 0.6976 - val_accuracy: 0.8030 - val_loss: 0.6636
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9445 - loss: 0.1771 - val_accuracy: 0.9751 - val_loss: 0.0928
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9609 - loss: 0.1317 - val_accuracy: 0.9515 - val_loss: 0.1487
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9701 - loss: 0.0957 - val_accuracy: 0.9788 - val_loss: 0.0712
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 18ms/step - accuracy: 0.9743 - loss: 0.0850 - val_accuracy: 0.9859 - val_loss: 0.0538
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9727 - loss: 0.0983 - val_accuracy: 0.8901 - val_loss: 0.4449
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9789 - loss: 0.0708 - val_accuracy: 0.9822 - val_loss: 0.0561
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9819 - loss: 0.0615 - val

In [24]:
performance = MHART.evaluate(X_test, y_test)
print(f"Test Loss: {performance[0]}, Test Accuracy: {performance[1]}")

311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9895 - loss: 0.0357
Test Loss: 0.033871620893478394, Test Accuracy: 0.9891238808631897


In [25]:
columns = ['Accel X (g)', 'Accel Y (g)', 'Accel Z (g)',
           'Gyro X (°/s)', 'Gyro Y (°/s)', 'Gyro Z (°/s)', 'Activity_Label']
df = pd.read_csv("Final_Resampled_HAR_And_Inertia.csv", header=0, names=columns)

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
df['Activity_Label'] = LE.fit_transform(df['Activity_Label'])

In [26]:
features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 6

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.7, random_state=21)

MHART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 275ms/step - accuracy: 0.8749 - loss: 0.8382

I0000 00:00:1721354018.188111  202244 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721354018.761224  202236 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721354019.740998  202241 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_3266', 148 bytes spill stores, 148 bytes spill loads



187/187 ━━━━━━━━━━━━━━━━━━━━ 59s 320ms/step - accuracy: 0.8750 - loss: 0.8361 - val_accuracy: 0.8647 - val_loss: 0.5283
Epoch 2/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.9270 - loss: 0.2201 - val_accuracy: 0.9584 - val_loss: 0.1462
Epoch 3/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9644 - loss: 0.1246 - val_accuracy: 0.9762 - val_loss: 0.0788
Epoch 4/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9744 - loss: 0.0794 - val_accuracy: 0.9869 - val_loss: 0.0494
Epoch 5/200
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.9789 - loss: 0.0755 - val_accuracy: 0.9842 - val_loss: 0.0462


In [27]:
performance_new = MHART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")

1087/1087 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8680 - loss: 0.5122
Test Loss: 0.5187253952026367, Test Accuracy: 0.867292046546936


# Mobile HART Trained on HAR and Inertia dataset finetuning on 80% of the Data

In [28]:
MHART = model.mobileHART_XS(input_shape, activityCount)

X_train, X_test, y_train, y_test = train_test_split(reshaped_data, window_labels, test_size=0.2, random_state=42)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = MHART.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stopping])

Epoch 1/200
  1/621 ━━━━━━━━━━━━━━━━━━━━ 11:54:42 69s/step - accuracy: 0.1094 - loss: 2.3080

I0000 00:00:1721354183.729171  198586 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_slice_fusion_3', 4 bytes spill stores, 4 bytes spill loads



621/621 ━━━━━━━━━━━━━━━━━━━━ 132s 102ms/step - accuracy: 0.6962 - loss: 0.7306 - val_accuracy: 0.7144 - val_loss: 0.7912
Epoch 2/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9424 - loss: 0.1857 - val_accuracy: 0.8773 - val_loss: 0.3834
Epoch 3/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9633 - loss: 0.1226 - val_accuracy: 0.9764 - val_loss: 0.0795
Epoch 4/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9684 - loss: 0.1015 - val_accuracy: 0.9832 - val_loss: 0.0567
Epoch 5/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9749 - loss: 0.0849 - val_accuracy: 0.9740 - val_loss: 0.0854
Epoch 6/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9737 - loss: 0.0850 - val_accuracy: 0.9702 - val_loss: 0.0993
Epoch 7/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9750 - loss: 0.0876 - val_accuracy: 0.9753 - val_loss: 0.0771
Epoch 8/200
621/621 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.9781 - loss: 0.0720 - va

In [29]:
features = df.columns[:-1]  
labels_encoded = df['Activity_Label'].values

data_filtered = df[features].values

window_size = 128
step_size = 64
num_features = 6

reshaped_data_filtered = create_overlapping_windows(data_filtered, window_size, step_size, num_features)

window_labels_filtered = assign_labels_to_windows(labels_encoded, window_size, step_size)

X_fine_tune, X_test_new, y_fine_tune, y_test_new = train_test_split(reshaped_data_filtered, window_labels_filtered, test_size=0.2, random_state=21)

MHART.fit(X_fine_tune, y_fine_tune, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/200
496/497 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.9219 - loss: 0.4175

I0000 00:00:1721354446.187211  202927 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 40 bytes spill stores, 40 bytes spill loads

I0000 00:00:1721354446.465592  202929 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 32 bytes spill stores, 32 bytes spill loads

I0000 00:00:1721354446.569497  202927 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 32 bytes spill stores, 32 bytes spill loads

I0000 00:00:1721354446.663470  202924 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_32195', 48 bytes spill stores, 48 bytes spill loads

I0000 00:00:1721354446.847789  202920 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721354449.583460

497/497 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step - accuracy: 0.9219 - loss: 0.4170

I0000 00:00:1721354502.199142  203107 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_8', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721354502.601811  203109 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_20', 16 bytes spill stores, 16 bytes spill loads

I0000 00:00:1721354505.193755  203109 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_3266', 4 bytes spill stores, 4 bytes spill loads



497/497 ━━━━━━━━━━━━━━━━━━━━ 88s 177ms/step - accuracy: 0.9220 - loss: 0.4165 - val_accuracy: 0.9739 - val_loss: 0.1202
Epoch 2/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9712 - loss: 0.1016 - val_accuracy: 0.6548 - val_loss: 2.9844
Epoch 3/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9747 - loss: 0.0828 - val_accuracy: 0.9869 - val_loss: 0.0442
Epoch 4/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9805 - loss: 0.0624 - val_accuracy: 0.9846 - val_loss: 0.0507
Epoch 5/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9773 - loss: 0.0750 - val_accuracy: 0.7216 - val_loss: 1.1055
Epoch 6/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9808 - loss: 0.0607 - val_accuracy: 0.9856 - val_loss: 0.0511
Epoch 7/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9817 - loss: 0.0595 - val_accuracy: 0.9812 - val_loss: 0.0547
Epoch 8/200
497/497 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9823 - loss: 0.0557 - val_accura

In [30]:
performance_new = MHART.evaluate(X_test_new, y_test_new)
print(f"Test Loss: {performance_new[0]}, Test Accuracy: {performance_new[1]}")

311/311 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9847 - loss: 0.0488
Test Loss: 0.05061814561486244, Test Accuracy: 0.9844914674758911


In [8]:
import tensorflow as tf
from model import mobileHART_XS  

input_shape = (128, 6)
activityCount = 10
MHART = mobileHART_XS(input_shape, activityCount)

MHART.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

converter = tf.lite.TFLiteConverter.from_keras_model(MHART)
tflite_model = converter.convert()

with open('mobilehart_model.tflite', 'wb') as f:
    f.write(tflite_model)
    
print("Model conversion to TensorFlow Lite format complete.")

I0000 00:00:1721836896.322692  246561 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721836896.322888  246561 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721836896.343774  246561 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721836896.343988  246561 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

INFO:tensorflow:Assets written to: /tmp/tmpxae3cqkx/assets


INFO:tensorflow:Assets written to: /tmp/tmpxae3cqkx/assets


Saved artifact at '/tmp/tmpxae3cqkx'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 128, 6), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  139794762444640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794736951360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139793615685584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794762442704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794011560064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794736953296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794011587152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794011588208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794011561472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794011586976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139794736959552: T

W0000 00:00:1721836905.262781  246561 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1721836905.262809  246561 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-07-24 12:01:45.263242: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpxae3cqkx
2024-07-24 12:01:45.275117: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-07-24 12:01:45.275130: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpxae3cqkx
2024-07-24 12:01:45.382869: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-07-24 12:01:45.401489: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-07-24 12:01:46.020365: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpxae3cqkx
2024-07-24 12:01:46.203096: I tensorflow/cc/saved_model/loader.cc

Model conversion to TensorFlow Lite format complete.
